In [14]:
import warnings
warnings.filterwarnings('ignore')

# Keras Application 응용
- 4가지 모델의 결과를 종합하여 결과 도출 ensemble 열에 추가

In [1]:
from glob import glob
glob('animals/*')

['animals\\dolphin.jpg',
 'animals\\eagle.jpg',
 'animals\\elephant.jpg',
 'animals\\hawk.jpg',
 'animals\\lion.jpg',
 'animals\\panda.jpg',
 'animals\\penguin.jpg',
 'animals\\rabbit.jpg',
 'animals\\shark.jpg',
 'animals\\sparrow.jpg']

In [2]:
#파일명에 해당이름만 가져옴
name_list = []
for file in glob('animals/*'):
    name = file.split('\\')[1]
    name = name.split('.')[0]
    name_list.append(name)
print(name_list)

['dolphin', 'eagle', 'elephant', 'hawk', 'lion', 'panda', 'penguin', 'rabbit', 'shark', 'sparrow']


## All models and ensemble

In [10]:
from tensorflow.keras.applications.vgg16 import VGG16, decode_predictions
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet import ResNet101
from tensorflow.keras.applications.resnet import ResNet152

import numpy as np 
from PIL import Image

In [11]:
vgg16 = VGG16()
vgg19 = VGG19()
resnet50 = ResNet50()
resnet101 = ResNet101()
resnet152 = ResNet152()

242900992/242900224 [==============================] - 85s 0us/step


In [15]:
vgg16_list, vgg19_list, resnet50_list, resnet101_list,resnet152_list = [],[],[],[],[]
ensemble = []

for file in glob('animals/*'):
    img = np.array(Image.open(file).resize((224,224)))
    yhat_v16 = vgg16.predict(img.reshape(-1, 224, 224, 3))
    yhat_v19 = vgg19.predict(img.reshape(-1, 224, 224, 3))
    yhat_r50 = resnet50.predict(img.reshape(-1, 224, 224, 3))
    yhat_r101 = resnet101.predict(img.reshape(-1, 224, 224, 3))
    yhat_r152 = resnet152.predict(img.reshape(-1, 224, 224, 3))
    yhat = ( yhat_v16[0]+yhat_v19[0]+yhat_r50[0]+yhat_r101[0]+yhat_r152[0] )/4.

    vgg16_list.append(decode_predictions(yhat_v16)[0][0][1])
    vgg19_list.append(decode_predictions(yhat_v19)[0][0][1])
    resnet50_list.append(decode_predictions(yhat_r50)[0][0][1])
    resnet101_list.append(decode_predictions(yhat_r101)[0][0][1])
    resnet152_list.append(decode_predictions(yhat_r152)[0][0][1])
    ensemble.append(decode_predictions(yhat.reshape(1,-1))[0][0][1])

## 데이터 프레임으로 합치기

In [16]:
import pandas as pd
df = pd.DataFrame({
    'Original':name_list, 'VGG16':vgg16_list, 'VGG19':vgg19_list,
    'ResNet50':resnet50_list, 'ResNet101':resnet101_list, 'ResNet152':resnet152_list,
    'Ensemble':ensemble
})
df

,Original,VGG16,VGG19,ResNet50,ResNet101,ResNet152,Ensemble
0,dolphin,sea_lion,dugong,sea_lion,king_penguin,sea_lion,sea_lion
1,eagle,kite,kite,kite,kite,kite,kite
2,elephant,African_elephant,African_elephant,African_elephant,African_elephant,African_elephant,African_elephant
3,hawk,kite,kite,kite,kite,kite,kite
4,lion,white_wolf,Arctic_fox,kuvasz,clumber,white_wolf,Arctic_fox
5,panda,gibbon,giant_panda,indri,giant_panda,giant_panda,giant_panda
6,penguin,king_penguin,king_penguin,king_penguin,king_penguin,king_penguin,king_penguin
7,rabbit,hare,hare,hare,hare,hare,hare
8,shark,great_white_shark,great_white_shark,great_white_shark,great_white_shark,great_white_shark,great_white_shark
9,sparrow,junco,house_finch,chickadee,indigo_bunting,indigo_bunting,chickadee
